In [1]:
cd /home/ubuntu/11777/src/baselines/vqa/

/home/ubuntu/11777/src/baselines/vqa


In [2]:
import json

In [66]:
WRITE = False

## VQA IDs

In [3]:
train_answer_file = 'data/v2_mscoco_train2014_annotations.json'
train_answers = json.load(open(train_answer_file))['annotations']

val_answer_file = 'data/v2_mscoco_val2014_annotations.json'
val_answers = json.load(open(val_answer_file))['annotations']

train_question_file = 'data/v2_OpenEnded_mscoco_train2014_questions.json'
train_questions = json.load(open(train_question_file))['questions']

val_question_file = 'data/v2_OpenEnded_mscoco_val2014_questions.json'
val_questions = json.load(open(val_question_file))['questions']

In [11]:
ta_ids = [x['image_id'] for x in train_answers]
tq_ids = [x['image_id'] for x in train_questions]
va_ids = [x['image_id'] for x in val_answers]
vq_ids = [x['image_id'] for x in val_questions]

len(ta_ids), len(tq_ids), len(va_ids), len(vq_ids)

(443757, 443757, 214354, 214354)

## Refexp IDs

In [12]:
cd /home/ubuntu/11777/src/baselines/refexp/Google_Refexp_toolbox/

/home/ubuntu/11777/src/baselines/refexp/Google_Refexp_toolbox


In [13]:
import scipy
%matplotlib inline
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
import matplotlib.patches as mpatches
import sys
import os

# Import Refexp python class
# Please MAKE SURE that ./google_refexp_py_lib is in your
# python library search path
sys.path.append("google_refexp_py_lib")
from refexp import Refexp

In [14]:
# Specify datasets path.
refexp_filename_train='google_refexp_dataset_release/google_refexp_train_201511_coco_aligned.json'
coco_filename_train='external/coco/annotations/instances_train2014.json'
refexp_filename_val='google_refexp_dataset_release/google_refexp_val_201511_coco_aligned.json'
coco_filename_val='external/coco/annotations/instances_val2014.json'

# Create Refexp instance.
refexp_train = Refexp(refexp_filename_train, coco_filename_train)
refexp_val = Refexp(refexp_filename_val, coco_filename_val)

loading annotations into memory...
Done (t=16.80s)
creating index...
index created!
Dataset loaded.
loading annotations into memory...
Done (t=9.55s)
creating index...
index created!
Dataset loaded.


In [23]:
len(refexp_train.imgIds), len(refexp_val.imgIds)

(24698, 4650)

## Common IDs

In [24]:
common_train_ids = set(refexp_train.imgIds).intersection(set(tq_ids))
common_val_ids = set(refexp_val.imgIds).intersection(set(vq_ids))

len(common_train_ids), len(common_val_ids)

(24698, 0)

In [25]:
n_heldout = 100
heldout_ids = np.random.permutation(list(common_train_ids))[:n_heldout]
heldout_ids

array([207360, 138201, 432526, 404812, 101467, 386203, 235098, 389731,
       412281,  68576, 529647, 111067, 518335, 308180, 243909, 525636,
       200971, 568963, 288610, 544410, 566700, 217043, 447016, 446726,
        36663, 114561, 137730,  88613, 496457,  66337, 435523, 147278,
       517534, 328995, 216757,  83149, 573975, 499545, 177193, 371435,
       551733, 132516, 281237, 406666, 146510, 485777, 172655, 352886,
        37433,  25105,  23702, 425859, 318953, 170119, 427803,  28038,
       274758, 415728, 164102, 525555, 265536, 189838, 426997, 192574,
       557434,  30065, 427238,  26939,  60999,  19374, 244646, 226232,
       517879, 570019, 286349, 531337, 238712, 328917, 227318, 364701,
       220991,  34389, 207231, 147369, 497274, 351239, 280199, 384953,
       256278, 220972, 569602, 124739, 152531, 537475,  44687,  81485,
       560587,  88210, 197951, 334596])

In [27]:
len([i for i in heldout_ids if i in tq_ids])

100

## Save revised train files

### VQA

In [30]:
cd /home/ubuntu/11777/src/baselines/vqa/

/home/ubuntu/11777/src/baselines/vqa


In [46]:
train_answer_file = 'data/v2_mscoco_train2014_annotations.json'
train_answers = json.load(open(train_answer_file))

train_question_file = 'data/v2_OpenEnded_mscoco_train2014_questions.json'
train_questions = json.load(open(train_question_file))

val_answer_file = 'data/v2_mscoco_val2014_annotations.json'
val_answers = json.load(open(val_answer_file))

val_question_file = 'data/v2_OpenEnded_mscoco_val2014_questions.json'
val_questions = json.load(open(val_question_file))

train_answer_file_filtered = 'data/v2_mscoco_train2014_annotations_filtered.json'
train_question_file_filtered = 'data/v2_OpenEnded_mscoco_train2014_questions_filtered.json'
val_answer_file_heldout = 'data/v2_mscoco_val2014_annotations_heldout.json'
val_question_file_heldout = 'data/v2_OpenEnded_mscoco_val2014_questions_heldout.json'

In [47]:
heldout_ids_set = set(heldout_ids)

val_answers['annotations'] = [x for x in train_answers['annotations'] if x['image_id'] in heldout_ids_set]
val_questions['questions'] = [x for x in train_questions['questions'] if x['image_id'] in heldout_ids_set]

train_answers['annotations'] = [x for x in train_answers['annotations'] if x['image_id'] not in heldout_ids_set]
train_questions['questions'] = [x for x in train_questions['questions'] if x['image_id'] not in heldout_ids_set]

In [48]:
ta_ids = [x['image_id'] for x in train_answers['annotations']]
tq_ids = [x['image_id'] for x in train_questions['questions']]
va_ids = [x['image_id'] for x in val_answers['annotations']]
vq_ids = [x['image_id'] for x in val_questions['questions']]

len(ta_ids), len(tq_ids), len(va_ids), len(vq_ids)

(443283, 443283, 474, 474)

In [50]:
if WRITE:
    with open(train_answer_file_filtered, 'w') as fp:
        json.dump(train_answers, fp)

    with open(train_question_file_filtered, 'w') as fp:
        json.dump(train_questions, fp)

    with open(val_answer_file_heldout, 'w') as fp:
        json.dump(val_answers, fp)

    with open(val_question_file_heldout, 'w') as fp:
        json.dump(val_questions, fp)

In [51]:
train_answer_file = 'data/v2_mscoco_train2014_annotations_filtered.json'
train_answers = json.load(open(train_answer_file))['annotations']

val_answer_file = 'data/v2_mscoco_val2014_annotations_heldout.json'
val_answers = json.load(open(val_answer_file))['annotations']

train_question_file = 'data/v2_OpenEnded_mscoco_train2014_questions_filtered.json'
train_questions = json.load(open(train_question_file))['questions']

val_question_file = 'data/v2_OpenEnded_mscoco_val2014_questions_heldout.json'
val_questions = json.load(open(val_question_file))['questions']

ta_ids = [x['image_id'] for x in train_answers]
tq_ids = [x['image_id'] for x in train_questions]
va_ids = [x['image_id'] for x in val_answers]
vq_ids = [x['image_id'] for x in val_questions]

len(ta_ids), len(tq_ids), len(va_ids), len(vq_ids)

(443283, 443283, 474, 474)

### Refexp

In [53]:
cd /home/ubuntu/11777/src/baselines/refexp/Google_Refexp_toolbox/

/home/ubuntu/11777/src/baselines/refexp/Google_Refexp_toolbox


In [54]:
# Specify datasets path.
refexp_filename_train1='/home/ubuntu/11777/src/baselines/refexp/Google_Refexp_toolbox/google_refexp_dataset_release/google_refexp_train_201511_coco_aligned.json'
refexp_filename_val1='/home/ubuntu/11777/src/baselines/refexp/Google_Refexp_toolbox/google_refexp_dataset_release/google_refexp_val_201511_coco_aligned.json'

refexp_filename_train2='/home/ubuntu/11777/data/refexp/google_refexp_train_201511_coco_aligned_and_labeled.json'
refexp_filename_val2='/home/ubuntu/11777/data/refexp/google_refexp_val_201511_coco_aligned_and_labeled.json'

refexp_train1 = json.load(open(refexp_filename_train1))
refexp_val1 = json.load(open(refexp_filename_val1))
refexp_train2 = json.load(open(refexp_filename_train2))
refexp_val2 = json.load(open(refexp_filename_val2))

In [65]:
train_ids1 = [y['image_id'] for x, y in refexp_train1['annotations'].items()]
val_ids1 = [y['image_id'] for x, y in refexp_val1['annotations'].items()]
train_ids2 = [y['image_id'] for x, y in refexp_train2['annotations'].items()]
val_ids2 = [y['image_id'] for x, y in refexp_val2['annotations'].items()]

len(set(train_ids1)), len(set(val_ids1)), len(set(train_ids2)), len(set(val_ids2))

(24698, 4650, 24698, 4650)

In [67]:
refexp_filename_train = '/home/ubuntu/11777/data/refexp/google_refexp_train_201511_coco_aligned_and_labeled.json'
refexp_filename_val = '/home/ubuntu/11777/data/refexp/google_refexp_val_201511_coco_aligned_and_labeled.json'

refexp_filename_train_filtered = '/home/ubuntu/11777/data/refexp/google_refexp_train_201511_coco_aligned_and_labeled_filtered.json'
refexp_filename_val_heldout = '/home/ubuntu/11777/data/refexp/google_refexp_val_201511_coco_aligned_and_labeled_heldout.json'

refexp_train = json.load(open(refexp_filename_train))
refexp_val = json.load(open(refexp_filename_val))

In [70]:
filtered_annotations = {}
heldout_annotations = {}

for ann_id, ann in refexp_train['annotations'].items():
    if int(ann['image_id']) in heldout_ids_set:
        heldout_annotations[ann_id] = ann
    else:
        filtered_annotations[ann_id] = ann

refexp_train['annotations'] = filtered_annotations
refexp_val['annotations'] = heldout_annotations

In [71]:
if WRITE:
    with open(refexp_filename_train_filtered, 'w') as fp:
        json.dump(refexp_train, fp)
    with open(refexp_filename_val_heldout, 'w') as fp:
        json.dump(refexp_val, fp)

In [72]:
refexp_filename_train_filtered = '/home/ubuntu/11777/data/refexp/google_refexp_train_201511_coco_aligned_and_labeled_filtered.json'
refexp_filename_val_heldout = '/home/ubuntu/11777/data/refexp/google_refexp_val_201511_coco_aligned_and_labeled_heldout.json'

refexp_train_filtered = json.load(open(refexp_filename_train_filtered))
refexp_val_heldout = json.load(open(refexp_filename_val_heldout))

train_ids = [y['image_id'] for x, y in refexp_train_filtered['annotations'].items()]
val_ids = [y['image_id'] for x, y in refexp_val_heldout['annotations'].items()]

len(train_ids), len(val_ids), len(set(train_ids)), len(set(val_ids))

(44646, 174, 24598, 100)